In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('./dataset/train_ts_feature_added.csv')
df_train['datetime_iso'] = pd.to_datetime(df_train['datetime_iso'])

df_test = pd.read_csv('./dataset/test_ts_feature_added.csv')
df_test['datetime_iso'] = pd.to_datetime(df_test['datetime_iso'])
df_test_date = df_test['datetime_iso']

In [10]:
df_train.head()

,temp,d_point,feels,min_temp,max_temp,prssr,hum,wind_spd,wind_deg,rain_1h,clouds,day_of_the_week,month,season,yearly_mean,monthly_mean,weekly_mean
0,24.75,23.89,25.76,24.28,25.22,1012.0,95.0,0.82,320.0,0.00,100.0,0,1,1,0.373315,0.442561,0.518929
1,24.58,23.73,25.57,23.99,25.26,1012.0,95.0,0.96,338.0,0.00,100.0,0,1,1,0.373315,0.442561,0.518929
2,26.60,24.06,26.60,26.10,27.39,1012.0,86.0,1.22,339.0,0.00,99.0,0,1,1,0.373315,0.442561,0.518929
3,27.31,24.37,30.90,26.59,28.36,1012.0,84.0,1.08,342.0,0.13,94.0,0,1,1,0.373315,0.442561,0.518929
4,27.41,25.05,31.54,26.58,28.31,1011.0,87.0,0.86,336.0,0.34,100.0,0,1,1,0.373315,0.442561,0.518929


## Modelling using Pycaret:Catboost

In [4]:
df_train = df_train.drop(['datetime_iso'],axis = 1)
df_test = df_test.drop(['datetime_iso','rain_1h'],axis = 1)

In [5]:
from pycaret.regression import *
s = setup(df_train, target = 'rain_1h', session_id = 123)

,Description,Value
0,Session id,123
1,Target,rain_1h
2,Target type,Regression
3,Original data shape,"(341880, 17)"
4,Transformed data shape,"(341880, 17)"
5,Transformed train set shape,"(239315, 17)"
6,Transformed test set shape,"(102565, 17)"
7,Numeric features,16
8,Rows with missing values,0.7%
9,Preprocess,True


In [6]:
catboost = create_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4492,0.7887,0.8881,0.2162,0.3648,1.2578
1,0.4261,0.6620,0.8136,0.2107,0.3527,1.2550
2,0.4383,0.7388,0.8595,0.1820,0.3598,1.2188
3,0.4420,0.7443,0.8627,0.1917,0.3617,1.2035
4,0.4361,0.7169,0.8467,0.1933,0.3579,1.2179
5,0.4390,0.7146,0.8454,0.1747,0.3622,1.2414
6,0.4280,0.6739,0.8209,0.2073,0.3550,1.2550
7,0.4366,0.7131,0.8445,0.1914,0.3589,1.2568
8,0.4358,0.7047,0.8395,0.1910,0.3585,1.2534


In [7]:
predictions = predict_model(catboost, data=df_test)
predictions.head()

,temp,d_point,feels,min_temp,max_temp,prssr,hum,wind_spd,wind_deg,clouds,day_of_the_week,month,season,yearly_mean,monthly_mean,weekly_mean,prediction_label
0,26.59,23.660000,26.590000,26.020000,27.160000,1009.0,84.0,1.45,355.0,97.0,0,1,1,NaN,0.442561,NaN,0.453469
1,26.51,24.920000,26.510000,26.059999,28.040001,1009.0,91.0,1.67,351.0,95.0,0,1,1,NaN,0.442561,NaN,0.263741
2,28.68,25.709999,34.680000,28.030001,29.299999,1009.0,84.0,1.72,345.0,90.0,0,1,1,NaN,0.442561,NaN,1.098017
3,28.84,25.250000,34.509998,28.520000,29.080000,1008.0,81.0,1.49,339.0,91.0,0,1,1,NaN,0.442561,NaN,1.152798
4,29.75,24.620001,35.380001,29.309999,30.570000,1007.0,74.0,1.39,339.0,96.0,0,1,1,NaN,0.442561,NaN,0.655935


In [11]:
submission = pd.DataFrame()
submission['datetime_iso'] = df_test_date
submission['rain_1h'] = predictions['prediction_label']
submission.head()

,datetime_iso,rain_1h
0,2018-01-01 00:00:00+00:00,0.453469
1,2018-01-01 01:00:00+00:00,0.263741
2,2018-01-01 02:00:00+00:00,1.098017
3,2018-01-01 03:00:00+00:00,1.152798
4,2018-01-01 04:00:00+00:00,0.655935


In [14]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   datetime_iso  49368 non-null  datetime64[ns, UTC]
 1   rain_1h       49368 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 771.5 KB


In [16]:
submission.to_csv('./submission/submission_1.csv', index=False)